# 1. Installing Neccessary Packages

In [ ]:
!sudo apt install tesseract-ocr
!sudo apt install libtesseract-dev
!apt install poppler-utils
!pip install PyPDF2 pytesseract opencv-python pdf2image pysimplegui

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 3 newly installed, 0 to remove and 10 not upgraded.
Need to get 4,795 kB of archives.
After this operation, 15.8 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 tesseract-ocr-eng all 4.00~git24-0e00fe6-1.2 [1,588 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 tesseract-ocr-osd all 4.00~git24-0e00fe6-1.2 [2,989 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic/universe amd64 tesseract-ocr amd64 4.00~git2288-10f4998a-2 [218 kB]
Fetched 4,795 kB in 1s (4,587 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl

In [ ]:
!pip install spacy --upgrade
# You might need to restart the runtime after running this line by using "Ctrl+M"

In [ ]:
!python -m spacy download en_core_web_sm

# 2. Download Demo Files

In [ ]:
!git clone https://github.com/teovercesi/SHSG_WinterSchool_GroupProject_PDF_Files /content/pdfs
# changing working directory into the /content/pdf/ to be able to read from this directory
%cd pdfs/

# 3. Main Program



## 3.1 Importing modules

In [ ]:
import PyPDF2
import os
import spacy
import cv2
import pytesseract
from pdf2image import convert_from_path
from termcolor import colored
pytesseract.pytesseract.tesseract_cmd = r'/usr/bin/tesseract'

## 3.2 Surpress Warnings for Demo Puposes

In [ ]:
import warnings
warnings.filterwarnings('ignore')

## 3.3 Defining the Course Catalogues

In [ ]:
# The following lists are to be used for Keyword searching:
# de denotes the "Deutsch"- Track
# en denotes the "English" Track
# den denotes Tracks that are in both languages
# General lists to find info on course structures:
general_de = ["Pflichtveranstaltung", "Pflichtfach", "Pflichtwahl", "Wahlbereich", "Kontextstudium", "Fokusbereich",
              "Sprachen", "Skills", "REKO"]
general_de = [each_string.lower() for each_string in general_de]

general_en = ["Core studies", "Compulsory subjects", "Contextual Studies", "Languages", "Skills", "REKO"]
general_en = [each_string.lower() for each_string in general_en]

# Alternative Search Keywords:
# alt_list = ["Kurse", "Study", "Academics", "ECTS", "Kursname"]
# alt_list = [each_string.lower() for each_string in alt_list]

# Lists for the Business Students (According to the new Study Order:
# https://universitaetstgallen.sharepoint.com/sites/BachelorDE/SitePages/BWL-Curriculum.aspx)

bbwl_de = ["mikroökonomik", "makroökonomik", "statistik", "leadership & human resource management",
           "strategisches management", "management", "empirische sozialforschung", "marketing",
           "corporate finance", "finance", "accounting", "controlling", "auditing", "operations-management",
           "wirtschafts & und steuerrecht", "informatik für wirtschaftswissenschaftler",
           "methoden der informatik für wirtschaftswissenschaftler"]
bbwl_de = [each_string.lower() for each_string in bbwl_de]

bbwl_en = ["business and fiscal law", "introduction to operations management", "accounting", "controlling", "auditing",
           "corporate finance",
           "methods of informatics for business studies", "fundamentals of informatics for business studies",
           "Microeconomics", "Macroeconomics", "Statistics",
           "Leadership& Human Resource Management", "Strategic Management", "Social Science Research", "Marketing"]
bbwl_en = [each_string.lower() for each_string in bbwl_en]

# List for the International Affairs Students :
# https://universitaetstgallen.sharepoint.com/sites/BachelorDE/SitePages/BIA-Curriculum-und-Reform-O20.aspx
bia_den = ["Quantitative Methods", "Quantitative Methoden", "Mikroökonomik", "Microeconomics", "Comparative Politics",
           "Vergleichende Politikwissenschaft", "Politische Theorie",
           "Public Management", "Qualitative Methoden", "Qualitative Methods", "International Economics",
           "European Governance", "Regieren in Europa", "International Relations",
           "Macroeconomics", "Makroökonomik", "International Law", "Accounting, Controlling, Auditing"]

bia_den = [each_string.lower() for each_string in bia_den]

# Lists for the Economics Students:
# https://universitaetstgallen.sharepoint.com/sites/BachelorDE/SitePages/VWL-Ordnung%2019.aspx
bvwl_de = ["Data Handling: Cleaning, Import and Visualization", "Data Analytics: Statistik", "Makroökonomik",
           "Mikroökonomik", "Accounting, Controlling, Auditing",
           "Empirische Wirtschaftsforschung", "Makroökonomik", "Mikroökonomik"]

bvwl_de = [each_string.lower() for each_string in bvwl_de]

bvwl_en = ["Data Handling: Cleaning, Import and Visualization", "Data Analytics: Statistics", "Macroeconomics",
           "Microeconomics", "Accounting, Controlling, Auditing",
           "Empirical Economic Research", "Macroeconomics", "Microeconomics"]

bvwl_en = [each_string.lower() for each_string in bvwl_en]

# List for the Law & Economics Students:
# https://universitaetstgallen.sharepoint.com/sites/BachelorDE/SitePages/BLE-Austausch.aspx
ble_de = ["Internationales Recht", "Juristische Wahlfächer", "Integrationsleistungen", "Law & Economics Methode",
          "Law & Economics Methode 2"]

ble_de = [each_string.lower() for each_string in ble_de]

# List for the Law Students:
# https://universitaetstgallen.sharepoint.com/sites/BachelorDE/SitePages/BLaw-Austausch.aspx
blaw = ["Privatrecht-Wirtschaftsrecht", "Verwaltungsrecht IIÖffentliches Wirtschaftsrecht", "Öffentliches Recht",
        "Strafrecht BT", "Internat. Privatrecht ",
        "Völkerrecht", "Europarecht", "Rechtsgeschichte"]

blaw = [each_string.lower() for each_string in blaw]

# Moving on to the Masters programmes, there is no structure of specific courses that can be taken abroad. Therefore,
# Only a general list with all relevant study terms will be passed:
general_master_de = ["Fokusbereich", "Unabhängiger Wahlbereich", "Kontextstudium", "Skills", "Pflichtwahlkurse",
                     "Pflichtkurse"]

general_master_en = ["Independent electives", "Concentration Areas", "Contextual Studies", "Areas of Concentration",
                     "Skills"]

## 3.4 Recognizing the University Name with AI and creating a File List

In [ ]:
# implementing AI to extract from input string the name of the university
def getting_uni_name_with_ai_from_input(a_string):
    # here we are loading the already trained pipeline
    nlp = spacy.load("en_core_web_sm")
    doc = nlp(a_string)

    # preparing our output
    entities = []

    # if the machine recognizes words based on the trained model it will return the word recognized
    for word in doc.ents:
        entities.append(word)

    # we want the output to be a string containing the name of the university
    return entities[0]


# once we have the uni name we must look for those files saved with the uni name inside
def getting_file_name_from_uni_name(uni_name):
    # first we look for all the files contained in the directory where we stored all the exchange pdfs
    files = os.listdir('/content/pdfs/')
    # then we define a file to search for based on the university name we received as input
    # IMPORTANT: the pdf file must contain the name of the university in LOWER CASES!!!
    name_file_to_search = str(uni_name).lower()

    # file_to_find will be our output and so the name of the file relative to the university we looked for
    # file_to_find = ""
    output_files_list = []

    for file in files:
        if name_file_to_search in file:
            # file_to_find = file
            output_files_list.append(file)

    # here we have now the name of the file which is the input of our pdf conversion and search for keywords
    if output_files_list:
        return output_files_list
    else:
        return None

## 3.5 Converting the PDF Files into Strings to be read

In [ ]:
def convert_pdf_into_string(file_name):

    # first of all open the pdf file from the directory where it is stored path = os.path.realpath(
    # r"C:\Users\matte\PycharmProjects\SHSG_GroupProject_WinterSchool\Exchange Files\\" + file_name)
    pdf_file_object = open(file_name, "rb")
    # print("The document to be read is: " + str(file_name))

    # and making it readable by python
    reader = PyPDF2.PdfFileReader(pdf_file_object)

    # to create the loop and string we need some preparation
    num_pages = reader.numPages
    # print("The number of pages is: " + str(num_pages))
    count_pages = 0
    output_text = ""

    # the loop reads through all the pages and return the text in a single string
    while count_pages < num_pages:
        page_to_read = reader.getPage(count_pages)
        count_pages += 1
        output_text += page_to_read.extractText()
    pdf_file_object.close()

    # if we do not have an empty string after this step then we are good to go
    if output_text != "":
        output_text = output_text

    # otherwise we are facing probably a PDF where text is not recognizable. To solve this problem we first
    # convert the PDF into and image and then use an alternative method to read from it
    else:

        # this is where the conversion of the pdf into an image takes place so that we can read it
        pages = convert_from_path(file_name, 100)
        page_number = 1

        # we print one page at the time (get one image at the time) -> otherwise it doesn't work -> poppler is needed
        # in the process
        for page in pages:
            page_name = str(page_number) + "out.png"
            page.save(page_name, "PNG")
            page_number += 1

        # for each image/page that we have printed we extract the text and append it to our text output which is
        # an empty string at the beginning of the process
        for i in range(1, page_number):
            image_to_read = cv2.imread(str(i) + "out.png")
            output_text += pytesseract.image_to_string(image_to_read)

    # due to the fact that we might have some useless punctuations and we are only interested in keywords we clean
    # most of the punctuations
    characters_to_remove = [",", ".", "\n", "\n\n", "\t", "  ", "   ", ";", "(", ")"]
    for character in characters_to_remove:
        output_text = output_text.replace(character, " ")

    # print("\nThe text of your PDF is: ")
    # print(output_text)
    return output_text

## 3.6 Looking for Keywords from Catalaogues in Strings

In [ ]:
# here we are trying to see if any keyword contained in the catalogue is also in the string (which is our pdf
# conversion output)

def search_string_for_keyword(a_string, a_catalogue):
    list_keywords_found = []
    # keywords_count = 0
    for keyword in a_catalogue:
        keyword = keyword.replace(" ", "")
        if keyword.lower() in a_string.lower():
            list_keywords_found.append(keyword)
            # keywords_count += 1
        else:
            pass
    return list_keywords_found

## 3.7 Summarizing our Output - Creating the Lists of Courses Found

In [ ]:
# here we take as input the list of files containing the name of the university, the course catalogues of a specific
# field of study and the catalogue of general study-terms
def get_list_of_courses(files_list, course_catalogues_for_study_field, general):
    # course_list is the list of all the courses found in the pdf text
    course_list = []

    # general_list is the list of all the relevant study-terms
    general_list = []

    for i in files_list:
        text = convert_pdf_into_string(i)
        text = text.replace(" ", "")
        text = text.lower()

        # here we see if courses contained in each catalogue are reported in the text we get as input
        for catalogue in course_catalogues_for_study_field:
            courses_found = search_string_for_keyword(text, catalogue)
            course_list.append(courses_found)

        # here we see if study-terms contained in each catalogue (of study-terms here) are reported in the text we
        # get as input
        for item in general:
            study_terms_found = search_string_for_keyword(text, item)
            general_list.append(study_terms_found)

    # Making copies of the lists which have all the results
    course_list_copy = course_list
    general_list_copy = general_list

    # Cleaning up the list for final presentation:
    course_list = [x for x in course_list if x != []]
    course_list = [item for sublist in course_list for item in sublist]
    course_list = list(set(course_list))

    general_list = [x for x in general_list if x != []]
    general_list = [item for sublist in general_list for item in sublist]
    general_list = list(set(general_list))

    # Printing the overall results:
    print("")
    print("For the university '" + colored(str(u_name).upper(), "green") + "', the following "
          + colored("Courses", "blue") + " have been found:")
    print(course_list)
    print("As to general " + colored("Information about Accreditation", "magenta")
          + ", the following have been mentioned:")
    print(general_list)
    print("")

    # Ascertaining length of original lists
    course_list_length = len(course_list_copy)
    identity_list_length = len(general_list_copy)

    # Yielding the length of the files:
    files_list_lenght = len(files_list)

    # Getting the separator length:
    course_len = int(course_list_length / files_list_lenght)

    identity_len = int(identity_list_length / files_list_lenght)

    # Printing which courses are mentioned in which report:
    old_value_courses = 0
    old_value_study_terms = 0

    new_value_courses = course_len
    new_value_study_terms = identity_len
    pdf_read_count = 1

    for file in files_list:
        print("" + colored(str(pdf_read_count) + ". ", "red") + colored(file, "yellow")
              + " contains the following " + colored("Courses:", "blue"))
        pdf_read_count += 1

        if course_list_copy[old_value_courses:new_value_courses] != [[], []]:
            course_list_copy_temp = [x for x in course_list_copy[old_value_courses:new_value_courses] if x != []]
            course_list_copy_temp = [i for sub in course_list_copy_temp for i in sub]
            course_list_copy_temp = list(set(course_list_copy_temp))
            print(course_list_copy_temp)
        else:
            print(None)

        print("As to general " + colored("Information about Accreditation", "magenta") + ", the following have been mentioned:")
        if general_list_copy[old_value_study_terms:new_value_study_terms] != [[], []]:
            general_list_copy_temp = [x for x in general_list_copy[old_value_study_terms:new_value_study_terms] if
                                      x != []]
            general_list_copy_temp = [i for sub in general_list_copy_temp for i in sub]
            general_list_copy_temp = list(set(general_list_copy_temp))
            print(general_list_copy_temp)
        else:
            print(None)

        print("")

        # updating the values for the next file item
        old_value_courses = new_value_courses
        new_value_courses = new_value_courses + course_len

        old_value_study_terms = new_value_study_terms
        new_value_study_terms = new_value_study_terms + identity_len

## 3.8 Executing All the Functions

In [ ]:
# This is where the UI begins and we actually run what we developed

# first we use AI to get the university name out of the user input and we make a list of all the files containing
# the university name
print("Hey! How can I help you with? :)")
input_text = input("Please, write your question here: ")
print("")

try:
  u_name = getting_uni_name_with_ai_from_input(input_text)
  # f_name = getting_file_name_from_uni_name(u_name)
  files_list_names = getting_file_name_from_uni_name(u_name)



  if files_list_names is None:
      print(colored("No files found, did you misspell the University name?", "red"))
      exit()

  # here we ask the study field of the user
  else:
      print("Please enter whether you are a " + colored("Bachelor", "yellow") + " or a " + colored("Master", "yellow")
            + " Student.")
      seniority = input("Write here: ")
      print("")

      if seniority == "Bachelor":
          print("Please, tell me your study field")
          print("You can choose between" + colored(" Business", "yellow") + "," + colored(" Economics", "yellow")
                + "," + colored(" International Affairs", "yellow") + "," + colored(" Law & Economics", "yellow")
                + "," + colored(" Law", "yellow") + ".")
          study_field = input("Write here: ")
          course_catalogue = []
          if study_field == "Business":
              course_catalogue = [bbwl_de, bbwl_en]
          elif study_field == "Economics":
              course_catalogue = [bvwl_de, bvwl_en]
          elif study_field == "International Affairs":
              course_catalogue = [bia_den]
          elif study_field == "Law & Economics":
              course_catalogue = [ble_de]
          elif study_field == "Law":
              course_catalogue = [blaw]
          else:
              pass

          general_info = [general_en, general_de]
          get_list_of_courses(files_list_names, course_catalogue, general_info)

      elif seniority == "Master":
          print("Since specific Core electives " + colored("cannot be taken outside of UniSG", "red") +
                " we will return the Study Terms found in the text: ")
          print("")

          general_info_master = [general_master_de, general_master_en]
          master_course_catalogue = [[], []]
          get_list_of_courses(files_list_names, master_course_catalogue, general_info_master)
except IndexError:
  print(colored('Sorry, was not able to recognize the University Name - did you misspell it?', 'red'))


# THE END :)